## Provided Helper Functions

In [1]:
# Taken from http://web.stanford.edu/class/cs221/ Assignment #2 Support Code
def dotProduct(d1, d2):
    """
    @param dict d1: a feature vector represented by a mapping from a feature (string) to a weight (float).
    @param dict d2: same as d1
    @return float: the dot product between d1 and d2
    """
    if len(d1) < len(d2):
        return dotProduct(d2, d1)
    else:
        return sum(d1.get(f, 0) * v for f, v in d2.items())

def increment(d1, scale, d2):
    """
    Implements d1 += scale * d2 for sparse vectors.
    @param dict d1: the feature vector which is mutated.
    @param float scale
    @param dict d2: a feature vector.

    NOTE: This function does not return anything, but rather
    increments d1 in place. We do this because it is much faster to
    change elements of d1 in place than to build a new dictionary and
    return it.
    """
    for f, v in d2.items():
        d1[f] = d1.get(f, 0) + v * scale

In [2]:
import os
import numpy as np
import pickle
import random
import timeit

'''
Note:  This code is just a hint for people who are not familiar with text processing in python. There is no obligation to use this code, though you may if you like. 
'''


def folder_list(path,label):
    '''
    Input: path to where the data is located and label (1 or -1 depending on positive or negative)
    Output: list of file contents 
    '''
    filelist = os.listdir(path)
    review = []
    for infile in filelist:
        file = os.path.join(path,infile)
        r = read_data(file)
        r.append(label)
        review.append(r)
    return review

def read_data(file):
    '''
    Read each file into a list of strings. 
    Example:
    ["it's", 'a', 'curious', 'thing', "i've", 'found', 'that', 'when', 'willis', 'is', 'not', 'called', 'on', 
    ...'to', 'carry', 'the', 'whole', 'movie', "he's", 'much', 'better', 'and', 'so', 'is', 'the', 'movie']
    '''
    f = open(file)
    lines = f.read().split(' ')
    symbols = '${}()[].,:;+-*/&|<>=~" '
    words = map(lambda Element: Element.translate(None, symbols).strip(), lines)
    words = filter(None, words)
    return words
	
###############################################
######## YOUR CODE STARTS FROM HERE. ##########
###############################################

def shuffle_data():
    '''
    pos_path is where you save positive review data.
    neg_path is where you save negative review data.
    '''
    pos_path = "data/data/pos"
    neg_path = "data/data/neg"
	
    pos_review = folder_list(pos_path,1)
    neg_review = folder_list(neg_path,-1)
	
    review = pos_review + neg_review
    random.shuffle(review)
    return review

## Load Data

In [3]:
pos_path = "data/data/pos"
neg_path = "data/data/neg"
pos_review = folder_list(pos_path,1)
neg_review = folder_list(neg_path,-1)

## Train - Test Split

In [4]:
shuffle=shuffle_data()
train = shuffle[:1500]
test = shuffle[1500:]

## Make matrix of data + label

In [5]:
import collections
def counter(data):
    y = []
    X = []
    for i in range(len(data)):
        y.append(data[i][-1])
        X.append(collections.Counter(data[i][:-1]))
    return X, y

In [6]:
X_train,y_train = counter(train)
X_test, y_test = counter(test)

## Slow Pegasos Algo 

In [7]:
def pegasos(X, y, lambda_reg, max_epochs):
    t = 0
    w = dict()
    epoch = 0
    while epoch < max_epochs:
        epoch += 1
        for i in range(len(X)):
            t += 1
            stepSize = 1/(t*lambda_reg)
            if y[i]*dotProduct(w, X[i]) < 1: # if the entry multiplited by the dot prod of w,x[i], update gradient with product
                increment(w,-stepSize*lambda_reg,w)
                increment(w,stepSize*y[i],X[i])
            else:#otherwise, update weight with existing times step size and lambda
                increment(w,-stepSize*lambda_reg,w)
    return w

## Fast Pegasos Algo

In [8]:
def pegasos_but_faster(X, y, lambd, max_epochs):
    t = 1
    w = dict()
    s = 1
    epoch = 0
    while epoch < max_epochs:
        epoch += 1
        for i in range(len(X)):
            t += 1
            stepSize = 1/(t*lambd)
            s *= (1-stepSize*lambd)
            if s*y[i]*dotProduct(w, X[i]) < 1:
                increment(w,(stepSize*y_train[i])/s,X_train[i])
    increment(w,(s-1),w)
    return w

In [9]:
def compare(X, y, lambd, max_epochs): 
    print 'Slow Version'
    startTime = timeit.default_timer()
    w = pegasos(X_train, y_train, lambd, max_epochs)
    endTime = timeit.default_timer()
    print("Run Time (sec) = {0}".format(endTime - startTime))
    
    print '\nFast Version'
    startTime = timeit.default_timer()
    w = pegasos_but_faster(X_train, y_train, lambd, max_epochs)
    endTime = timeit.default_timer()
    print("Run Time (sec) = {0}".format(endTime - startTime))

## Get error and best lambda 

In [10]:
def calcError(w_list, X, y):
    score_list = [accuracy(i, X, y) for i in w_list]
    return score_list

def accuracy(w, X, y):
    count = 0
    for i in range(len(X)):
        if y[i]*dotProduct(w, X[i])<0:
            count += 1
    return count/(1. * len(X))

def getbest(loss, lambdas, weights):
    min_Loss = min(loss)
    i = loss.index(min_Loss)
    bestL = lambdas[i]
    bestW = weights[i]
    return bestL, bestW, min_Loss

## Run the algo on an array of lambdas 

In [11]:
def try_out_lambdas(X_train, y_train, lambda_reg_list, num_epochs):
    classificationScore_list, w_list = [], []
    startTime = timeit.default_timer()
    for i in lambda_reg_list:
        w = pegasos_but_faster(X_train, y_train, i, num_epochs)
        w_list.append(w)
        classificationScore_list.append(accuracy(w, X_train, y_train))
    endTime = timeit.default_timer()
    print("Run Time (sec) = {0}".format(endTime - startTime))
    return classificationScore_list, w_list

## Find words that were incorrectly classified

In [12]:
def mistake_finder(X, y, w):
    incorrect = []
    for i in range(len(y)):
        if y[i]*dotProduct(w, X[i])<0:
            incorrect.append(X[i])
    return incorrect

## Run It All!

In [13]:
def pegasos_runner(lambd_arr): 
    print "Prove that Fast is Fast"
    compare(X_train,y_train,0.1, 2)
    
    print "\nTraining"
    train_scores, w_list = try_out_lambdas(X_train,y_train,lambd_arr, 2)
    score_list = calcError(w_list, X_train, y_train)
    bestLambda, bestw, min_Loss = getbest(score_list, lambd_arr, w_list)
    

    print "\nTesting"
    mistakes = mistake_finder(X_test, y_test, bestw)
    print("{0} misclassified words".format(len(mistakes)))


In [14]:
pegasos_runner([0.1, 0.15, 0.2])

Prove that Fast is Fast
Slow Version
Run Time (sec) = 111.520427135

Fast Version
Run Time (sec) = 0.464596112879

Training
Run Time (sec) = 1.97334542096

Testing
114 misclassified words
